# Retention study analysis

## Data set preparation and XGboost Fit

Data set is built with retention_study_data_set_gen.ipynb

Create train/validation/test split on data. Right now doing single validation set, but could consider expanding to k-fold cross validation to tune one of the hyperparameters like scale positive weight or the prediction lead time. If we do this with lead time, this will require multiple versions of the data set to be built.



### Variable nomenclature

summary where * is train/test/val, N = number of month observations, F = number of features, P = number of physicians
| Variable | Description|
|----------|-------------|
| X_*       | (N x F) DataFrame of features|
| y_*       |(N x 1) Series of binary depart within interval|
| X_*_ids   |(N x 1) Series of physician ids for each month|
| *_ids     |(P x 2) Dataframe with [0,:] list of physician ids, [1,:] binary depart within study for physician |
| y_*_pred  |(N x1) series of binary predicitons |
| y_*_pred_prob | (Nx1) series of raw xgboost output |

## Import Data

In [ ]:
import datetime
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from matplotlib import pyplot as plt
import shap
from copy import deepcopy
import xgboost as xgb
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer, confusion_matrix, plot_confusion_matrix
from sklearn.metrics import classification_report, auc, roc_curve, precision_recall_curve, f1_score
from sklearn.base import clone as skClone
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

SEED = 0
np.random.seed(SEED)
pd.set_option('display.max_columns', None)
PATH = './data/processed/turbo_7_29_22_deid_processed_3_ROUND_5y_TENURE_NO_STUDYDAY.pkl'

In [ ]:
categorical_columns_impute = [
    'age_group', 
    'tenure', 
    'number_of_rx_errors',
    'EWA_avg_number_of_rx_errors'
]
numeric_columns_impute = ['physician_demand',
    'teamwork_on_inbox_value',
    'r_slope_note_quality_manual_value',
    'r_slope_note_time_8',
    'r_slope_order_time_8',
    'wow_time_8',
    'review_time_8',
    'r_slope_ib_time_8',
    'EWA_avg_wow_time_8',
    'r_slope_patient_volume',
    'risk_avg',
    'EWA_avg_note_quality_manual_value',
    'r_slope_panel_cnt',
    'r_slope_note_quality_contribution_value',
    'physician_work_intensity',
    'EWA_avg_risk_avg',
    'r_slope_ehr_time_8',
    'r_slope_review_time_8',
    'panel_cnt',
    'note_time_8',
    'r_slope_physician_work_intensity',
    'ib_time_8',
    'r_slope_physician_demand',
    'r_slope_teamwork_on_inbox_value',
    'EWA_avg_note_quality_contribution_value',
    'EWA_avg_panel_cnt',
    'EWA_avg_teamwork_on_inbox_value',
    'r_slope_risk_avg',
    'ehr_time_8',
    'EWA_avg_order_time_8',
    'note_quality_contribution_value',
    'EWA_avg_ib_time_8',
    'note_quality_manual_value',
    'EWA_avg_note_time_8',
    'r_slope_wow_time_8',
    'order_time_8',
    'r_slope_number_of_rx_errors'
    ]

In [ ]:
def get_pipe():
    # found here: https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html
    numeric_transformer = Pipeline(
        steps=[("imputer", SimpleImputer(strategy="median"))]
    )
    categorical_transformer = Pipeline(
        steps=[("imputer", SimpleImputer(strategy="most_frequent"))]
    )

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_columns_impute),
            ("cat", categorical_transformer, categorical_columns_impute),
        ],
        remainder='passthrough',
    )
    # pipeline for the model
    pipe = Pipeline(
        steps=[
            ("preprocessor", preprocessor),        
        ]
    )
    return pipe

In [ ]:
def load_data(path):
    categorical_cols = [
    'physician_id',
    'age_group',
    'gender',
    'departure_in_interval',
    'calendar_month',
    'covid_wave'
    ]
    ehr_data = pd.read_pickle(path)
    ehr_data = ehr_data.drop(['provtype_Physician','reportingperiodstartdate'],axis=1)
    mask = (ehr_data['specialty_Family Medicine']==0) & (ehr_data['specialty_Internal Medicine']==0) & (ehr_data['specialty_Pediatrics']==0)
    # Apply mask to the data so we can NaN the panel count and panel complexity on specialty that is not
    # specialty_Family, specialty_Internal, specialty_Pediatrics
    ehr_data.loc[mask, 'panel_cnt'] = np.nan
    ehr_data.loc[mask, 'risk_avg'] = np.nan
    all_ids = pd.DataFrame({
        'id':  ehr_data['physician_id'].unique(),
        'depart': ehr_data.groupby('physician_id')['departure_in_interval'].any().tolist()
    })
    ID_train, ID_test = train_test_split(ehr_data['physician_id'].unique(),
                                                test_size=0.2, random_state=SEED)
    assert len(set(ID_train).intersection(set(ID_test))) == 0, 'Bad split'
    X_train = ehr_data[ehr_data['physician_id'].isin(ID_train)].drop(['departure_in_interval', 'physician_id'], axis=1)
    X_test = ehr_data[ehr_data['physician_id'].isin(ID_test)].drop(['departure_in_interval', 'physician_id'], axis=1)
    y_train = ehr_data[ehr_data['physician_id'].isin(ID_train)]['departure_in_interval']
    y_test = ehr_data[ehr_data['physician_id'].isin(ID_test)]['departure_in_interval']
    X_save = ehr_data
    y_save = ehr_data.pop('departure_in_interval')
    X_save_ids = ehr_data.pop('physician_id')
    save_ids = all_ids
    
    return X_train, X_test, y_train, y_test, X_save, y_save, X_save_ids, save_ids

In [ ]:
def load_data2(path, impute=False):
    '''
    Looks like I was given some data that has been preprocessed and has the specialty mask already enabled, only issue
    is that I cant split based on id, so going to do my best guess
    '''
    categorical_cols = [
    'age_group',
    'gender',
    'departure_in_interval',
    'calendar_month',
    'covid_wave'
    ]
    X,y = pd.read_pickle(path)
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=SEED)
    if impute:
        pipe = get_pipe()
        X_train = pipe.fit_transform(X_train)
        X_test = pipe.transform(X_test)
    return X_train, X_test, y_train, y_test

In [ ]:
#X_train, X_test, y_train, y_test, X_save, y_save, X_save_ids, save_ids = load_data(PATH)
# This impute == true is to deal with data that has NaNs, same can be done for the 
# original loader but probs best to merge the two
X_train, X_test, y_train, y_test = load_data2('masked_for_kevin.pkl', impute=True)

### Model Performance Evaluation Script

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams["figure.figsize"] = (10,7)
def model_perf(classifier, X_train, X_test, y_train, y_test,crosstab=True, stats = True, roc_plot = True, optimal_thresh=True, custom_thresh = None, conf_interval=False, bootstrap_nsamples = 200):
    # simple helper to easily show some key features of performace
    X = X_test
    y = y_test
    y_pred = classifier.predict(X)
    probs = classifier.predict_proba(X)
    scores = probs[:,1]
    fpr, tpr, threshold = roc_curve(y, scores)
    roc_auc = auc(fpr, tpr)

    if crosstab:
        display(pd.crosstab(y,y_pred))
    
    if stats:
        print(classification_report(y, y_pred))
        print(get_stats(y, y_pred))

    # method I: plt
    if roc_plot:
        plt.title('Main Results ROC Curve and AUC')
        _ROC = plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
        #plt.legend(loc = 'lower right')
        plt.plot([0, 1], [0, 1],'r--')
        plt.xlim([0, 1])
        plt.ylim([0, 1])
        plt.ylabel('True Positive Rate')
        plt.xlabel('False Positive Rate')
        #plt.show()
    if optimal_thresh:
        opt_cutoff, ix = cutoff_youdens_j(fpr, tpr, threshold)#Find_Optimal_Cutoff(y, scores)[0]
        print('Optimal Threshold cutoff')
        #print(opt_cutoff)
        #ix = gmean(fpr, tpr, threshold)
        plt.plot(fpr[ix], tpr[ix], marker='o', color='black', label='Best Threshold (Youdens J Stat) =%f' % (opt_cutoff))
        #plt.text(2,4,'This text starts at point (2,4)')
        plt.vlines(fpr[ix], 0, 1, linestyles='dashed', color='black')
        print(classification_report(y, scores > opt_cutoff))
        display(pd.crosstab(y,scores > opt_cutoff))
        print(get_stats(y, scores > opt_cutoff))
    if custom_thresh is not None:
        for ct in custom_thresh:
            print('Custom Thresh')
            print(ct)
            print(classification_report(y, scores > ct))
            display(pd.crosstab(y,scores > ct))
            print(get_stats(y, scores > ct))
    if conf_interval:
        bootstrap_stats = bootstrap_auc(classifier, X_train, y_train, X_test, y_test, nsamples=bootstrap_nsamples)
        roc_aucs = []
        roc_aucs = []
        pr_aucs = []
        for i in range(len(bootstrap_stats)):
            roc_aucs.append(bootstrap_stats[i]['roc_auc'])
            pr_aucs.append(bootstrap_stats[i]['pr_auc'])
        roc_CI = np.percentile(roc_aucs, (2.5, 97.5))
        pr_CI = np.percentile(pr_aucs, (2.5, 97.5))
        plt.fill_between(fpr, (tpr-(roc_CI[1]- roc_CI[0])), (tpr+(roc_CI[1]- roc_CI[0])), alpha=0.2)
        _ROC[0].set_label(f'AUC = {roc_auc:.2f} CI=[{roc_CI[0]:.2f} - {roc_CI[1]:.2f}]')
    plt.legend(loc = 'lower right')
    plt.show()
def cutoff_youdens_j(fpr,tpr,thresholds):
    j_scores = tpr-fpr
    ix = np.argmax(j_scores)
    best_thresh = thresholds[ix]
    print('Best Threshold (Youdens J Stat)=%f' % (best_thresh))
    j_ordered = sorted(zip(j_scores,thresholds))
    return j_ordered[-1][1], ix

def gmean(fpr,tpr,thresholds):
    # calculate the g-mean for each threshold
    gmeans = np.sqrt(tpr * (1-fpr))
    # locate the index of the largest g-mean
    ix = np.argmax(gmeans)
    print('Best GMeans Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))
    return ix

def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
            TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
            FP += 1
        if y_actual[i]==y_hat[i]==0:
            TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
            FN += 1
    return(TP, FP, TN, FN)
def get_stats(true_value, classifier_output):
    # we need sensitivity, specificity, npv, ppv for all 3 thresholds
    # Note that in binary classification, recall of the positive class 
    # is also known as “sensitivity”; recall of the negative class is “specificity”.
    TN, FP, FN, TP = confusion_matrix(true_value, classifier_output).ravel() #perf_measure(true_value, classifier_output)# confusion_matrix(true_value, classifier_output).ravel()
    ppv = TP/(TP+FP)
    npv = TN/(TN+FN)
    specificity = TN/(TN+FP)
    sensitivity = TP/(TP+FN)
    return {'ppv': ppv, 'npv': npv, 'specificity': specificity, 'sensitivity': sensitivity}
    
# configure bootstrap
def bootstrap_auc(fit_clf, X_train, y_train, X_test, y_test, nsamples=1000):
    X_train = _numpy_to_df(X_train)
    y_train = _numpy_to_df(y_train)
    X_test = _numpy_to_df(X_test)
    #y_test = _numpy_to_df(y_test)
    statistics = {}
    for b in range(nsamples):
        clf = skClone(fit_clf)# gives us a unfit clone of the classifier
        idx = np.random.randint(X_train.shape[0], size=X_train.shape[0])
        clf.fit(X_train.iloc[idx], y_train.iloc[idx].values.ravel())# strange that we needed to ravel the values after? something to do with pandas and numpy
        pred = clf.predict_proba(X_test)[:, 1]
        #roc_auc = roc_auc_score(y_test.ravel(), pred.ravel())
        fpr, tpr, threshold = roc_curve(y_test.ravel(), pred.ravel())
        roc_auc = auc(fpr, tpr)
        precision, recall, thresholds = precision_recall_curve(y_test.ravel(), pred.ravel())
        pr_auc = auc(recall, precision)
        statistics[b] = {'roc_auc': roc_auc, 'fpr': fpr, 'tpr': tpr, 'precision': precision, 'recall': recall, 'pr_auc':pr_auc}
        
    return statistics#np.percentile(auc_values, (2.5, 97.5))
def _numpy_to_df(df):
    if type(df) == np.ndarray:
        return pd.DataFrame(deepcopy(df))
    return df

# Train models

In [ ]:
'''# dummy Data: 
from sklearn.datasets import make_classification
def get_dummy_data(random_state = 0):
    X, y = make_classification(n_samples=2000, n_features=25, random_state=random_state, class_sep=0.25)
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    return X_train, X_test, y_train, y_test'''

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
# import baseline models:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
def trainBaselines(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    model_pred = model.predict(X_test)
    model_prob = model.predict_proba(X_test)
    model_perf(model,X_train, X_test, y_train, y_test,stats=True,roc_plot=True, optimal_thresh=True, crosstab=True, conf_interval=True, bootstrap_nsamples=50)

# Baselines

## Baseline -- Logistic Regression

In [ ]:
LR = LogisticRegression(random_state=SEED)
#X_train, X_test, y_train, y_test = get_dummy_data()# THIS LINE SHOULD BE REMOVED TO USE REAL DATA!
trainBaselines(LR, X_train, X_test, y_train, y_test)

## Baseline -- Gaussian Naive Bayes

In [ ]:
gnb = GaussianNB()
#X_train, X_test, y_train, y_test = get_dummy_data()
trainBaselines(gnb, X_train, X_test, y_train, y_test)

## Baseline -- Decision Tree

In [ ]:
DTC = tree.DecisionTreeClassifier()
#X_train, X_test, y_train, y_test = get_dummy_data()
trainBaselines(DTC, X_train, X_test, y_train, y_test)

## Baseline -- Random Forest

In [ ]:
RFC = RandomForestClassifier(random_state=SEED)
#X_train, X_test, y_train, y_test = get_dummy_data()
trainBaselines(RFC, X_train, X_test, y_train, y_test)

## Baseline -- XGBOOST

In [ ]:
classify_xgb = xgb.XGBClassifier(
    objective = 'binary:logistic',
    seed = SEED,
)
#X_train, X_test, y_train, y_test = get_dummy_data()
trainBaselines(classify_xgb, X_train, X_test, y_train, y_test)

# GridsearchCV -- XGBOOST

In [ ]:
'''from sklearn.metrics import f1_score
import numpy as np

def f1_eval(y_pred, dtrain):
    y_true = dtrain.get_label()
    err = 1-f1_score(y_true, np.round(y_pred), average=None)
    return 'f1_err', err'''

In [ ]:
classify_xgb = xgb.XGBClassifier(
    objective = 'binary:logistic',
    #missing = nan,
    seed = SEED,
    scale_pos_weight = 400, # approx 
    n_estimators=200
)

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {#'nthread':[2], #when use hyperthread, xgboost may become slower
              'objective':['binary:logistic'],
              'learning_rate': [0.3, 0.4, 0.5], #so called `eta` value
              #'max_depth': [x for x in range(4, 10)],
              #'reg_lambda':[1,10,20,40]
            }


In [ ]:
classify_xgb_GS = GridSearchCV(classify_xgb, parameters, n_jobs=1, 
                                scoring='roc_auc',#make_scorer(f1_score, average='binary'),#'roc_auc',
                                cv=5,
                                verbose=2, refit=True)

In [ ]:
classify_xgb_GS.fit(X_train, y_train)

In [ ]:
print(classify_xgb_GS.best_params_)
print(classify_xgb_GS.best_score_)

In [ ]:
y_test_pred = classify_xgb_GS.predict(X_test)
y_test_pred_prob = classify_xgb_GS.predict_proba(X_test)

In [ ]:
#classify_xgb_save = classify_xgb_GS.best_estimator_.fit(X_train,y_train)
classify_xgb_save = classify_xgb_GS.best_estimator_

In [ ]:
model_perf(classify_xgb_GS,X_train, X_test, y_train, y_test,stats=True,roc_plot=True, optimal_thresh=True, crosstab=True)# custom_thresh=[0.8, 0.001]
# can add: conf_interval=True, bootstrap_nsamples=50) to get the band and the CI

In [ ]:
# Lets store results
import pickle
train_list = [classify_xgb_save,X_save,y_save,X_save_ids,save_ids]
fpath = './models/xgb_classifier_train_test_without_specialty_5y.pkl'
with open(fpath,"wb") as open_file:
    pickle.dump(train_list,open_file)

# ROC curve

In [ ]:
# Roc Curve
fpr, tpr, threshold = roc_curve(y_test, y_test_pred_prob[:,1])
roc_auc = auc(fpr, tpr)
# precision recall curve
precision, recall, thresholds = precision_recall_curve(y_test, y_test_pred_prob[:,1])
# calculate precision-recall AUC
pr_auc = auc(recall, precision)

In [ ]:
bootstrap_stats = bootstrap_auc(classify_xgb_GS.best_estimator_, X_train, y_train, X_test, y_test, nsamples=200)

In [ ]:
roc_aucs = []
pr_aucs = []
for i in range(len(bootstrap_stats)):
    roc_aucs.append(bootstrap_stats[i]['roc_auc'])
    pr_aucs.append(bootstrap_stats[i]['pr_auc'])
roc_CI = np.percentile(roc_aucs, (2.5, 97.5))
pr_CI = np.percentile(pr_aucs, (2.5, 97.5))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(15,5))
ax1.set_title('Receiver Operating Characteristic')
ax1.plot(fpr, tpr, 'b', label = 'AUC = {:.2f} [CI={:.3f},{:.3f}]'.format(roc_auc, roc_CI[0], roc_CI[1]))
#plt.plot(bootstrap_stats[maxIDX]['fpr'], bootstrap_stats[maxIDX]['tpr'], 'r')
#plt.plot(bootstrap_stats[minIDX]['fpr'], bootstrap_stats[minIDX]['tpr'], 'r')
ax1.fill_between(fpr, (tpr-(roc_CI[1]- roc_CI[0])), (tpr+(roc_CI[1]- roc_CI[0])), alpha=0.2)
ax1.legend(loc = 'lower right')
ax1.plot([0, 1], [0, 1],'r--')
ax1.set_xlim([0, 1])
ax1.set_ylim([0, 1])
ax1.set_ylabel('True Positive Rate')
ax1.set_xlabel('False Positive Rate')
ax2.set_title('Precision Recall Curve')
ax2.plot(recall, precision, 'b', label = 'AUC = {:.2f} [CI={:.3f},{:.3f}]'.format(pr_auc, pr_CI[0], pr_CI[1]))
# fill between
ax2.fill_between(recall, (precision-(pr_CI[1]- pr_CI[0])), (precision+(pr_CI[1]- pr_CI[0])), alpha=0.2)
# calculate the no skill line as the proportion of the positive class
no_skill = len(y_test[y_test==1]) / len(y_test)# Essentially the fraction of positive classes/total number of examples
# plot the no skill precision-recall curve
ax2.plot([0, 1], [no_skill, no_skill], 'r--', label='No Skill = %0.2f' % no_skill)
ax2.legend(loc = 'best')
ax2.set_xlim([0, 1])
ax2.set_ylim([0, 1])
ax2.set_xlabel('Recall')
ax2.set_ylabel('Precision')
plt.show()

In [ ]:
ehr_data_ = pd.read_pickle('./data/processed/turbo_7_29_22_deid_processed_3_ROUND_5y_TENURE_NO_STUDYDAY.pkl')

In [ ]:
len(ehr_data_[ehr_data_['departure_in_interval'] == True]['physician_id'].unique())

In [ ]:
44/len(ehr_data_['physician_id'].unique())

# END